In [5]:
import numpy as np
from Library import readdata
from Library import readdata_time
from Library import evaluation_F1
from Library import evaluation_NDCG
from Library import save_result
from Library import read_feature
from numpy import *
#from LatentFactor import LatentFactorModel
import xlwt
import time
import json
import tensorflow as tf

In [6]:
##parameter setting
dataset = 5                         # Datasets selecting 0 to 5 for 'All', '_Women', '_Men', '_CLothes', '_Shoes', '_Jewelry' respectively
eta = 0.03                          # learning rate
I = 200                             # length of latent feature
J = 100                             # length of latent feature
top_k = [5, 10, 20, 50, 100]        # number to recommend
batch_size_train = 5000             # batch size for traing
batch_size_test = 1000              # batch size for tessing
lambda_c = 0.1                      # weighting parameter for couple matrices
lambda_r = 1.5                      # regularization coefficient
vali_test = 0                       # 0 for validate set,1 for test set
feat = [3]                          # feature selecting, 0 for CNN, 1 for AES, 2 for CH, 3 for CNN+AES
feature_length = 1000               # length of feature
#epoch = 200                         # number to iteration
epoch = 5
sample_rate = 5                     # sample sample_rate negative samples for each positive item

In [7]:
def get_feature(dataset):
    # to load features
    feat_list = ['CNN', 'AES', 'CH', 'CNN_AES']             # feature list
    F = read_feature(feat_list[feat[0]], dataset, Q)
    for i in range(1, len(feat)):
        F = np.hstack((F, read_feature(feat_list[feat[i]], dataset, Q)))
    return F


In [8]:
def get_order(Order, length):
    # combining several recommendation lists into one, for tensor-based model
    order = []
    ind = 0
    while len(order) <= length:
        for line in Order:
            if not line[ind] in order:
                order.append(line[ind])
        ind += 1
    return order

In [9]:
def test_DCFA(U, Vu, Vt, T, M, N, F):
    # test the effectiveness
    U = mat(U)
    Vu = mat(Vu)
    Vt = mat(Vt)
    T = mat(T)
    F = mat(F)
    M = mat(M)
    N = mat(N)
    k_num = len(top_k)
    # k_num-long lists to record F1 and NDCG
    F1 = np.zeros(k_num)
    NDCG = np.zeros(k_num)
    num_item = len(Test)

    # choose batch_size_test test samples randomly
    for i in range(batch_size_test):
        j = int(math.floor(num_item * random.random()))
        # test data: [u, [i, i, i, i], [r, r, r]], where u, i, r are for user, item, time, respectively
        u = Test[j][0]
        test_item = Test[j][1]
        # score for all users
        Order = []
        for r in Test[j][2]:
            # for each r, score all items
            UV = U[u] * Vu.T + M[u] * F.T
            VT = T[r] * Vt.T + N[r] * F.T
            UV = np.array(UV.tolist()[0])
            VT = np.array(VT.tolist()[0])
            score = (UV * VT).tolist()
            # order
            b = zip(score, range(len(score)))
            b = sorted(b, key=lambda x: x[0])
            order = [x[1] for x in b]
            order.reverse()
            Order.append(order)
        # train samples
        train_positive = train_data_aux[u][0]
        # we have len(train_data_aux[u][1]) k-length recommendation lists for each user,
        # to compare fairly with other baselines, we combine len(train_data_aux[u][1]) k-length list to one k-length lists for each user
        # we will remove at most len(train_positive) train samples from order, so we return k+len(train_positive) items
        order = get_order(Order, top_k[-1] + len(train_positive))
        # remove the train samples from the recommendations
        for item in train_positive:
            try:
                order.remove(item)
            except:
                continue
        # we also remove the train samples from test samples
        test_item = list(set(test_item) - set(train_positive))
        # test F1 and NDCG for each k
        for i in range(len(top_k)):
            F1[i] += evaluation_F1(order, top_k[i], test_item)
            NDCG[i] += evaluation_NDCG(order, top_k[i], test_item)
    # calculate the average
    F1 = (F1 / batch_size_test).tolist()
    NDCG = (NDCG / batch_size_test).tolist()
    return F1, NDCG

In [11]:
from tensorflow.keras.activations import sigmoid

In [12]:
def train_DCFA(eta):
    # train the model
    # initialization
    U = np.array([np.array([(random.random() / math.sqrt(I)) for j in range(I)]) for i in range(P)])
    Vu = np.array([np.array([(random.random() / math.sqrt(I)) for j in range(I)]) for i in range(Q)])
    Vt = np.array([np.array([(random.random() / math.sqrt(J)) for j in range(J)]) for i in range(Q)])
    T = np.array([np.array([(random.random() / math.sqrt(J)) for j in range(J)]) for i in range(R)])

    M = np.array([np.array([(random.random() / math.sqrt(K)) for j in range(K)]) for i in range(P)])
    N = np.array([np.array([(random.random() / math.sqrt(K)) for j in range(K)]) for i in range(R)])
    e = 10**10

    # output a result without training
    print('iteration ', 0,)
    [F1, NDCG] = test_DCFA(U, Vu, Vt, T, M, N, F)
    Fmax = 0
    if F1[0] > Fmax:
        Fmax = F1[0]
    print(Fmax, 'F1: ', F1, '  ', 'NDCG1: ', NDCG)
    # save to the .xls file
    save_result([' '], [''] * len(top_k), [''] * len(top_k), path_excel)
    save_result('metric', ['F1'] * len(top_k), ['NDCG'] * len(top_k), path_excel)
    save_result('Top_k', top_k, top_k, path_excel)
    save_result([' '], [''] * len(top_k), [''] * len(top_k), path_excel)
    save_result('iteration ' + str(0), F1, NDCG, path_excel)

    # the number of train samples
    Re = len(train_data)
    # split the train samples with a step of batch_size_train
    bs = list(range(0, Re, batch_size_train))
    bs.append(Re)

    for ep in range(0, epoch): # while not converged && iter < iter_max do
        print('iteration ', ep + 1,)
        eta = eta * 0.99
        # iterate all train samples in one epoch
        for i in range(0, len(bs) - 1):
        # for each batch do
            if abs(U.sum()) < e:
                # initialize dU and dC to record the gradient
                dU = np.zeros((P, I))
                dVu = np.zeros((Q, I))
                dVt = np.zeros((Q, J))
                dT = np.zeros((R, J))

                dM = np.zeros((P, K))
                dN = np.zeros((R, K))
                for re in range(bs[i], bs[i + 1]):
                # for each record in current batch do
                    # train sample: [u, i, r]
                    p = train_data[re][0]
                    qi = train_data[re][1]
                    r = train_data[re][2]

                    UV = np.dot(U[p], Vu[qi])
                    VT = np.dot(Vt[qi], T[r])
                    MDF = np.dot(M[p], F[qi])
                    NEF = np.dot(N[r], F[qi])

                    Bi = UV + MDF
                    Ci = VT + NEF
                    Ai = Bi * Ci

                    num = 0
                    # choose sample_rate negative items, and calculate the gradient
                    while num < sample_rate:
                    # select 5 non-observed items q′ randomly
                        qj = int(random.uniform(0, Q))
                        if (not qj in train_data_aux[p][0]) and (not qj in train_time_aux[r][1]):
                            num += 1
                            UV = np.dot(U[p], Vu[qj])
                            VT = np.dot(Vt[qj], T[r])
                            MDF = np.dot(M[p], F[qj])
                            NEF = np.dot(N[r], F[qj])

                            Bj = UV + MDF
                            Cj = VT + NEF
                            Aj = Bj * Cj

                            Bij = Bi - Bj
                            Cij = Ci - Cj
                            Aij = Ai - Aj

                            dU[p] += sigmoid(-Aij) * (Ci * Vu[qi] - Cj * Vu[qj]) + lambda_c * sigmoid(-Bij) * (Vu[qi] - Vu[qj])
                            dVu[qi] += sigmoid(-Aij) * Ci * U[p] + lambda_c * sigmoid(-Bij) * U[p]
                            dVu[qj] -= sigmoid(-Aij) * Cj * U[p] + lambda_c * sigmoid(-Bij) * U[p]
                            dM[p] += sigmoid(-Aij) * (Ci * F[qi] - Cj * F[qj]) + lambda_c * sigmoid(-Bij) * (F[qi] - F[qj])
                            dVt[qi] += sigmoid(-Aij) * Bi * T[r] + lambda_c * sigmoid(-Cij) * T[r]
                            dVt[qj] -= sigmoid(-Aij) * Bj * T[r] + lambda_c * sigmoid(-Cij) * T[r]
                            dT[r] += sigmoid(-Aij) * (Bi * Vt[qi] - Bj * Vt[qj]) + lambda_c * sigmoid(-Cij) * (Vt[qi] - Vt[qj])
                            dN[r] += sigmoid(-Aij) * (Bi * F[qi] - Bj * F[qj]) + lambda_c * sigmoid(-Cij) * (F[qi] - F[qj])

                # update the matrices
                U += eta * (dU - lambda_r * U)
                Vu += eta * (dVu - lambda_r * Vu)
                Vt += eta * (dVt - lambda_r * Vt)
                T += eta * (dT - lambda_r * T)
                M += eta * (dM - lambda_r * M)
                N += eta * (dN - lambda_r * N)

        if abs(U.sum()) < e:
            [F1, NDCG] = test_DCFA(U, Vu, Vt, T, M, N, F)
            if F1[0] > Fmax:
                Fmax = F1[0]
            print(Fmax, 'F1: ', F1, '  ', 'NDCG1: ', NDCG)
            save_result('iteration ' + str(ep + 1), F1, NDCG, path_excel)
        else:
            break
        #return U, Vu, Vt, T, M, N
    if abs(U.sum()) < e:
        return 0
    else:
        return 1

In [13]:
def save_parameter():
    # record the parameters
    dataset_list = ['all', '_Women', '_Men', '_CLothes', '_Shoes', '_Jewelry']
    excel = xlwt.Workbook()
    table = excel.add_sheet('A Test Sheet')
    table.write(0, 0, 'model')
    table.write(0, 2, 'DCFA')
    table.write(1, 0, 'dataset')
    table.write(1, 2, dataset_list[dataset])
    table.write(2, 0, 'eta')
    table.write(2, 2, eta)
    table.write(3, 0, 'I')
    table.write(3, 2, I)
    table.write(4, 0, 'J')
    table.write(4, 2, J)
    table.write(5, 0, 'top_k')
    for i in range(len(top_k)):
        table.write(5, 2 + i, top_k[i])
    table.write(6, 0, 'batch_size_train')
    table.write(6, 2, batch_size_train)
    table.write(7, 0, 'batch_size_test')
    table.write(7, 2, batch_size_test)
    table.write(8, 0, 'lambda_c')
    table.write(8, 2, lambda_c)
    table.write(9, 0, 'lambda_r')
    table.write(9, 2, lambda_r)
    table.write(10, 0, 'vali_test')
    table.write(10, 2, vali_test)
    table.write(11, 0, 'feat')
    for i in range(len(feat)):
        table.write(11, 2 + i, feat[i])
    table.write(12, 0, 'fea_len')
    table.write(12, 2, feature_length)
    table.write(13, 0, 'epoch')
    table.write(13, 2, epoch)
    table.write(17, 0, ' ')

    excel.save(path_excel)

In [14]:
def print_parameter():
    print('model', 'DCFA')
    print('dataset', dataset)
    print('eta', eta)
    print('I', I) 
    print('J', J) 
    print('top_k', top_k)
    print('batch_size_train', batch_size_train)
    print('batch_size_test', batch_size_test)
    print('lambda_c', lambda_c)
    print('lambda_r', lambda_r)
    print('vali_test', vali_test) 
    print('feat', feat) 
    print('feature_length', feature_length) 
    print('epoch', epoch)
    print

In [ ]:
'''*************************main function****************************'''
'''*************************main function****************************'''
for i in range(1):
    # datasets
    dataset_list = ['', '_Women', '_Men', '_CLothes', '_Shoes', '_Jewelry']
    # load data
    [train_data, train_data_aux, validate_data, test_data, P, Q] = readdata(dataset_list[dataset])
    # load data for tensor factorization
    [train_record_aux, train_time_aux, R] = readdata_time(dataset_list[dataset])
    # load features
    F = get_feature(dataset_list[dataset])
    K = len(F[0])
    # select test set or validation set
    if vali_test == 0:
        Test = validate_data
    else:
        Test = test_data

    #for j in range(1):
    path_excel = 'experiment_result/' + dataset_list[dataset] + '_DCFA_' + str(int(time.time())) + str(int(random.uniform(100,900))) + '.xls'
    save_parameter()
    print_parameter()
    train_DCFA(eta)

model DCFA
dataset 5
eta 0.03
I 200
J 100
top_k [5, 10, 20, 50, 100]
batch_size_train 5000
batch_size_test 1000
lambda_c 0.1
lambda_r 1.5
vali_test 0
feat [3]
feature_length 1000
epoch 5
iteration  0
0.0006190476190476103 F1:  [0.0006190476190476103, 0.001047785547785533, 0.0008358742706568667, 0.000935171746492489, 0.0009258883220395879]    NDCG1:  [0.000470365282770005, 0.0005919064335373652, 0.00048700744085603193, 0.0004926911363325911, 0.0004815457923808024]
iteration  1


2022-02-18 16:22:47.751771: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


0.010494322344322187 F1:  [0.010494322344322187, 0.010239332889332726, 0.008074121997165331, 0.005368427199857196, 0.003817693112923568]    NDCG1:  [0.008170980716041583, 0.0070044296368740626, 0.005503662068618826, 0.003785994025856993, 0.002754895196102557]
iteration  2
